In [51]:
import torch
from torchvision import datasets, transforms
from torchvision.transforms import ToTensor
from torch import nn
from torch.utils.data import DataLoader, Subset
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np

In [53]:
transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5,), (0.5,)),])

In [67]:
# Download training data from open datasets.
training_data = datasets.MNIST(
    root="data",
    train=True,
    download=True,
    transform=transform,
)

# Download test data from open datasets.
test_data = datasets.MNIST(
    root="data",
    train=False,
    download=True,
    transform=transform,
)

In [ ]:
#Segregate the data of 28*28 images into different classes
segregated_data = {}
for X,y in training_data:
    if segregated_data.get(y) is None:
        segregated_data[y] = np.empty((0,28,28))
    segregated_data[y] = np.concatenate((segregated_data[y], X), axis=0)

In [119]:
#Converting to useful form
seg_dataset = []

for i in range(10):
    for j in range(len(segregated_data[i])):
        seg_dataset.append((torch.tensor(segregated_data[i][j], dtype=torch.float32),i))

In [121]:
batch_size = 64

train_dataloader = DataLoader(seg_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=batch_size, shuffle=True)


In [122]:
# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")


Using mps device


In [123]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [124]:
lr = 1e-2
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=lr)

In [128]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [129]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [130]:
epochs = 20
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.305882  [   64/60000]
loss: 1.936262  [ 6464/60000]
loss: 1.429415  [12864/60000]
loss: 0.812439  [19264/60000]
loss: 0.702639  [25664/60000]
loss: 0.491097  [32064/60000]
loss: 0.460886  [38464/60000]
loss: 0.511725  [44864/60000]
loss: 0.846774  [51264/60000]
loss: 0.417299  [57664/60000]
Test Error: 
 Accuracy: 88.5%, Avg loss: 0.395947 

Epoch 2
-------------------------------
loss: 0.442581  [   64/60000]
loss: 0.191852  [ 6464/60000]
loss: 0.472289  [12864/60000]
loss: 0.184826  [19264/60000]
loss: 0.284960  [25664/60000]
loss: 0.635566  [32064/60000]
loss: 0.343021  [38464/60000]
loss: 0.274967  [44864/60000]
loss: 0.485783  [51264/60000]
loss: 0.374833  [57664/60000]
Test Error: 
 Accuracy: 90.4%, Avg loss: 0.329777 

Epoch 3
-------------------------------
loss: 0.432985  [   64/60000]
loss: 0.388994  [ 6464/60000]
loss: 0.218833  [12864/60000]
loss: 0.216253  [19264/60000]
loss: 0.173990  [25664/60000]
loss: 0.264265  [32064/600

For full data the accuracy after 20 epochs is 97.2
80% data = 97.8
60% data = 97.7
40% data = 98
20% data = 98.1